
## Set up and imports

## Load the dataset catalog
Read datasets.json into memory; normalize to a dict keyed by dataset name for quick access.

## Helper to show description, time range, bounds, variables, etc

## Loader registry (bridges catalog → dataset.py)

## Pick a dataset and preview it.

## LLM → plotting code
Let users ask “plot monthly mean SST over Arabian Sea for 2015” and have the notebook generate runnable code that uses the already-loaded object

## Plot helpers for maps: time-series & snapshot
Fallbacks if the LLM struggles

In [ ]:
# Example helpers
# 1) makes a time series by averaging the variable across all non-time dimensions
def plot_timeseries_mean(ds, var, time_slice=None): # ds: xarray.Dataset var: variable name 
    da = ds[var]
    if time_slice is not None:
        da = da.sel(time=slice(*time_slice))
    ts = da.mean(dim=[d for d in da.dims if d!="time"]) # ts: 1-D over time
    ts.plot()
    plt.title(f"{var} global mean over time")
    plt.tight_layout()

# 2) plots a single time slice as a 2-D map
def plot_snapshot_map(ds, var, time=None):
    da = ds[var]
    if time is not None:
        da = da.sel(time=time) # da: 2-D; if the exact timestamp isn’t present, may use .sel(time=time, method="nearest")
    da.plot()
    plt.title(f"{var} snapshot")
    plt.tight_layout()
